# Import the necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nltk
#from nltk.stem.lancaster import LancasterStemmer
#stemmer = LancasterStemmer()
#nltk.download('punkt')  Punkt Sentence Tokenizer. This tokenizer divides a text into a list of sentences
#by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences.
#nltk.download('wordnet') WordNet is a lexical database for the English language, which was created by Princeton,and is
#part of the NLTK corpus. You can use WordNet alongside the NLTK module to find the meanings of words, synonyms and more.
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
import numpy
import tflearn
import tensorflow
import random
import json
import pickle
import csv

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








# Load the dataset

In [4]:
with open("diseases.json") as file:
    data = json.load(file)

# Data pre-processing

In [5]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [lemmatizer.lemmatize(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [lemmatizer.lemmatize(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


In [6]:
symptoms=["hi","hello","bye","thanks","whats up","Hey"]
with open("diseases1.csv") as file1:
    symp = csv.reader(file1, delimiter=',')
    for row in symp:
        for s in row:
            symptoms.append(s)

# Create a DNN model

In [7]:
tensorflow.reset_default_graph() #Clears the default graph stack and resets the global default graph.

#We are building a 3-layers neural network using TFLearn. We need to specify the shape of our input data. In our case,
#each sample has a total of 318 features and we will process samples per batch to save memory, so our data 
#input shape is [None, 318] ('None' stands for an unknown dimension, so we can change the total number of samples 
#that are processed in a batch).
net = tflearn.input_data(shape=[None, len(training[0])]) #to know length of features we use len(training[0])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") #Softmax function is used in the output layer of a 
                                    #neural net to represent a probability distribution of possible outcomes of the network.
net = tflearn.regression(net) #used in TFLearn to apply a regression (linear or logistic) to the provided input. It requires
                              #to specify a TensorFlow gradient descent optimizer 'optimizer' that will minimize the provided 
                              #loss function 'loss'

model = tflearn.DNN(net) #TFLearn provides a model wrapper 'DNN' that can automatically performs a neural network
                         #classifier tasks, such as training, prediction, save/restore, etc...


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# Fit and save the model

In [8]:
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True) #We will run it for 1000 epochs (the network 
                                                                #will see all data 1000 times) with a batch size of 8.
    
    model.save("model.tflearn")  # acc: 0.5106 -- iter: 401/401 for 1000 epochs
                                 # acc: 0.4652 -- iter: 401/401 for 1500 epochs

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Kranthi Kumar\chatBotMajorProject_YT\model.tflearn


# Create function to give input to the Model.

In [9]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

# Create GUI Chat window and other necessary components with tkinter

In [10]:
#Creating GUI with tkinter
import tkinter
from tkinter import *

# Function to remove special chars and symbols

In [11]:
import re
def remove_special(m):
    m1=m.replace(","," ")
    nestr = re.sub(r'[^a-zA-Z0-9 ]',r'',m1)
    return nestr

# Function to get output from the model

In [12]:
def chatbot_response(msg):
    results = model.predict([bag_of_words(remove_special(msg), words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if(results[results_index]>0.7):
        for tg in data["intents"]:
            if tg['tag'] == tag:
                t=tg['tag']
                responses = tg['responses']
                syms=tg["patterns"]
                info=",".join(syms[1:])
        
        if t not in ["greetings","name","goodbye","thanks","options","diseases"]:
            return random.choice(responses),info
        else:
            return random.choice(responses),""
    else:
        return "Sorry,I'm unable to figure out.Please add some more symptoms.",""


In [13]:
l=[]
dis=["brain hemorrhage","piles","BP","Acne","Alcoholic hepatitis","Allergy","Arthritis","Benign Paroxymsal Positional Vertigo","Bronchial Asthma","COVID-19","Cervical spondylosis","Chicken pox","Chronic cholestasis","Common Cold","Dengue,Diabetes","Dimorphic hemmorhoids(piles)","Drug Reaction","Fungal infection","GERD","Gastroenteritis","Heart attack","Hepatitis A","Hepatitis B","Hepatitis C","Hepatitis D","Hepatitis E","Hypertension","Hyperthyroidism","Hypoglycemia","Hypothyroidism","Impetigo","Jaundice","Malaria","Migraine","Osteoarthristis","Paralysis (brain hemorrhage)","Peptic ulcer disease","Pneumonia","Psoriasis","Tuberculosis","Typhoid","Urinary tract infection","Varicose veins","fever"]
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    s=msg
    s=s.replace(" ","")
    
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "You: " + msg + '\n\n')
    ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    if s=="":
        ChatLog.insert(END, "DocBot: " + "You entered nothing.Please enter your symptoms." + '\n\n')
    
    elif s.isnumeric():
        ChatLog.insert(END, "DocBot: " + "You entered number.Please enter your symptoms." + '\n\n')
    
    elif s in ".,!@#$%^&*()_+={[}];:<>?":
        ChatLog.insert(END, "DocBot: " + "You entered a special symbol.Please enter your symptoms." + '\n\n')
 
    else:
        res,add_info=chatbot_response(msg)

        if res=="Sorry,I'm unable to figure out.Please add some more symptoms.":
            if s.isalnum():
                flag=0
                for i in symptoms:
                    if(msg.lower()==i.lower()):
                        l.append(msg)
                        r=",".join(l)
                        res,add_info=chatbot_response(r)
                        if res=="Sorry,I'm unable to figure out.Please add some more symptoms.":
                            ChatLog.insert(END, "DocBot: " + res + '\n\n')
                        else:
                            ChatLog.insert(END, "DocBot: " + res + '\n\n')
                            entered_symptoms=",".join(l)
                            #ChatLog.insert(END, "DocBot: Symptoms you entered : " + entered_symptoms + '\n\n')
                            ChatLog.insert(END, "DocBot: It's symptoms include : " + add_info + '\n\n')
                            percentage=len(list(entered_symptoms.split(",")))*100/len(list(add_info.split(",")))
                            ChatLog.insert(END, "DocBot: I am " + str(percentage) + "% confident that you may have this disease." '\n\n')       
                            l.clear()
                        flag=1
                        break
                if flag==0:
                    ChatLog.insert(END, "DocBot: " + "You entered alphanumeric.Try again with the symptoms I have known." + '\n\n')
            else:
                ChatLog.insert(END, "DocBot: " + "Sorry,I can't understand you.Try again entering your symptoms." + '\n\n')

                
        else:
            ChatLog.insert(END, "DocBot: " + res + '\n\n')
            if add_info!="":
                f=0
                for i in dis:
                    if(msg.lower()==i.lower()):
                        ChatLog.insert(END, "DocBot: It's symptoms include : " + add_info + '\n\n')
                        f=1
                        break
                if(f==0):
                    l.append(msg)
                    entered_symptoms=",".join(l)
                    #ChatLog.insert(END, "DocBot: Symptoms you entered : " + entered_symptoms + '\n\n')
                    ChatLog.insert(END, "DocBot: It's symptoms include : " + add_info + '\n\n')
                    percentage=len(list(entered_symptoms.split(",")))*100/len(list(add_info.split(",")))
                    ChatLog.insert(END, "DocBot: I am " + str(percentage) + "% confident that you may have this disease." '\n\n')
            l.clear()

    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)
    
    if msg.lower()=="quit":
        l.clear()
        base.destroy() 

In [ ]:
base = Tk()
base.title("DocBot")
base.geometry("400x500")
base.resizable(width=TRUE, height=TRUE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

In [19]:
import anvil.server
anvil.server.connect("N3VOJ64VKF5RPRCK5FSL44OG-XCKWEJIILBT3FLCU")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [20]:
@anvil.server.callable

def DocBot(msg):
    import nltk
    from nltk.stem import WordNetLemmatizer
    import re
    lemmatizer = WordNetLemmatizer()
    
    def remove_special(m):
        m1=m.replace(","," ")
        nestr = re.sub(r'[^a-zA-Z0-9 ]',r'',m1)
        return nestr
    
    def bag_of_words(s, words):
        bag = [0 for _ in range(len(words))]
        s_words = nltk.word_tokenize(s)
        s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]
        for se in s_words:
            for i, w in enumerate(words):
                if w == se:
                    bag[i] = 1
        return numpy.array(bag)
                
    results = model.predict([bag_of_words(remove_special(msg), words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if(results[results_index]>0.7):
        for tg in data["intents"]:
            if tg['tag'] == tag:
                t=tg['tag']
                responses = tg['responses']
                syms=tg["patterns"]
                info=","
                info=info.join(syms[1:])
        if t not in ["greetings","name","goodbye","thanks","options","diseases"]:
            return random.choice(responses),info
        else:
            return random.choice(responses),""
    else:
        return "Sorry,I'm unable to figure out.Please add some more symptoms.",""

In [21]:
anvil.server.get_app_origin()

'https://docbot.anvil.app'